In [2]:
#ライブラリのインポート
import TkEasyGUI as sg  #
import os
import chardet          #
import pandas as pd     #
import re
import math
import openpyxl         #

In [9]:
import datetime

dt_now = datetime.datetime.now()
dt_now.year, dt_now.month, dt_now.day

(2024, 7, 25)

In [1]:
# masterデータが存在すればそれを返し、存在しなければ新たに作成し、保存する
def checkMasterCSV(file_path) :

    # ファイルの存在確認
    if os.path.exists(file_path):
        # ファイルが存在すれば読み込む
        #！！！readerrorとpermissionerrorでtryを通すこと！！！
        masterdf = pd.read_csv(file_path, index_col=0, dtype={'受給者証番号':'str', '月間アセスメント担当者名':'str'})
        
    else:
        # ファイルが存在しなければ新しいデータフレームを作成
        columns = ['受給者証番号', '月間アセスメント担当者名']
        masterdf = pd.DataFrame(columns=columns)
        
        # フォルダが無ければ作成
        # 相対パスにファイル名のみを指定した場合、フォルダ名は''となる
        if os.path.dirname(file_path) != '' :
            if os.path.exists(os.path.dirname(file_path)) == False :
                os.makedirs(os.path.dirname(file_path))
            # データフレームをcsvファイルとして保存
        masterdf.to_csv(file_path, index=True)

    return masterdf

In [3]:
# ファイルパスを指定
file_path = "usermaster.csv"

# ファイルの存在確認
masterdf = checkMasterCSV(file_path=file_path)

In [60]:
os.path.dirname(file_path)

''

In [6]:
masterdf

,受給者証番号,月間アセスメント担当者名
test,00001234,taro


In [5]:
masterdf.loc['test'] = ['00001234', 'taro']
masterdf.to_csv(file_path, index=True)

In [48]:
searchword = 'test2'

# Key列をキーとしてデータフィールド内を検索
if searchword in masterdf.index.values:
    # 存在する場合、その行の他の列の要素を取得
    result_row = masterdf[masterdf.index == searchword]
    print(result_row.values[0][0])
    print(result_row.values[0][1])
else:
    # 存在しない場合、新たな行を追加
    default_value1 = 1555
    default_value2 = 'dayu'
    masterdf.loc[searchword] = [default_value1, default_value2]
    # 追加されたデータフレームを再びCSVファイルに保存
    masterdf.to_csv(file_path, index=True)

1555
dayu


In [47]:
result_row.values[0][0]

1555

In [2]:
import calendar, datetime

year = 2024
month = 3

#0→月曜日　6→日曜日
dt = datetime.datetime(year, month, 1)
n = dt.weekday()
cell_list_startplace = n
#nは5以上になると7を引き、-2から再びカウントが進む
#nが0以上の時にリストに追加する
if n > 4 :
    n -= 7
    cell_list_startplace = 0

#月末日を求める
lastdate = datetime.date(year, month, calendar.monthrange(year, month)[1]).day

day_list = []

for i in range(1, lastdate, 1) :
    if n >= 0 :
        day_list.append(i)

    n += 1
    if n > 4 : n -= 7

day_list

[1, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 25, 26, 27, 28, 29]

In [3]:
## ['B11', 'B13', 'B15', 'B17', 'B19', 'O11', 'O13', ... , 'O19', 'B28', 'B30', ...]

column_list = ['B', 'C', 'E', 'F', 'G', 'I', 'O', 'P', 'R', 'S', 'T', 'V']
#row_listは必ずintで扱うこと！！
row_list = list(range(11, 21, 2))*2 + list(range(28, 38, 2))*2 + list(range(45, 55, 2))*2
cell_list = []

for i, r in enumerate(row_list) :
    #B, B, B, B, B, O, O, O, O, O, B, B, ...
    column = 'B'
    if int(i / 5) % 2 == 1 :
        column = 'O'
    
    cell_list.append([column_list[column_list.index(column)  ] + str(r),
                    column_list[column_list.index(column) + 1] + str(r),
                    column_list[column_list.index(column) + 2] + str(r),
                    column_list[column_list.index(column) + 2] + str(r + 1),
                    column_list[column_list.index(column) + 3] + str(r),
                    column_list[column_list.index(column) + 3] + str(r + 1),
                    column_list[column_list.index(column) + 4] + str(r),
                    column_list[column_list.index(column) + 5] + str(r)])

## cell_list[入力箇所0~29][日ごとの入力欄0~7]

In [4]:
import jpbizday  #
#jpbizdayは営業日を拾ってくるライブラリ

#データのある日付リストも別途作成し、
#2つのリストを照合して(inを用いて)存在しない場合は「予定休」→欠席に○を付ける

for i, d in enumerate(day_list) :
    if jpbizday.is_bizday(datetime.date(year, month, d)) :
        print(d, cell_list[i + cell_list_startplace][:])
    else :
        print(d, "祝日")

1 ['B19', 'C19', 'E19', 'E20', 'F19', 'F20', 'G19', 'H19']
4 ['O11', 'P11', 'R11', 'R12', 'S11', 'S12', 'T11', 'U11']
5 ['O13', 'P13', 'R13', 'R14', 'S13', 'S14', 'T13', 'U13']
6 ['O15', 'P15', 'R15', 'R16', 'S15', 'S16', 'T15', 'U15']
7 ['O17', 'P17', 'R17', 'R18', 'S17', 'S18', 'T17', 'U17']
8 ['O19', 'P19', 'R19', 'R20', 'S19', 'S20', 'T19', 'U19']
11 ['B28', 'C28', 'E28', 'E29', 'F28', 'F29', 'G28', 'H28']
12 ['B30', 'C30', 'E30', 'E31', 'F30', 'F31', 'G30', 'H30']
13 ['B32', 'C32', 'E32', 'E33', 'F32', 'F33', 'G32', 'H32']
14 ['B34', 'C34', 'E34', 'E35', 'F34', 'F35', 'G34', 'H34']
15 ['B36', 'C36', 'E36', 'E37', 'F36', 'F37', 'G36', 'H36']
18 ['O28', 'P28', 'R28', 'R29', 'S28', 'S29', 'T28', 'U28']
19 ['O30', 'P30', 'R30', 'R31', 'S30', 'S31', 'T30', 'U30']
20 祝日
21 ['O34', 'P34', 'R34', 'R35', 'S34', 'S35', 'T34', 'U34']
22 ['O36', 'P36', 'R36', 'R37', 'S36', 'S37', 'T36', 'U36']
25 ['B45', 'C45', 'E45', 'E46', 'F45', 'F46', 'G45', 'H45']
26 ['B47', 'C47', 'E47', 'E48', 'F47', '

In [4]:
#ファイルを読み込み、読み込むファイルのエンコードを判別する
#改めてファイルを読み込み、データフレームにする
#それぞれのデータフレームの氏名欄に\u3000(全角スペース)がある場合、半角スペースに変換する

#この3つを同時に行い、データフィールドを戻り値とする関数
def csvConvert(file_path) :
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    encoding = result['encoding']
    
    df = pd.read_csv(file_path, encoding=encoding)

    if '\u3000' in df.columns.values[0]:
        df = df.rename(columns={df.columns.values[0] : df.columns.values[0].replace('\u3000',' ')})

    return df

In [3]:
#フォーマットとなるExcelファイルの存在確認
try:
    wb = openpyxl.load_workbook('サービス支援記録ーSample Format(張翔).xlsx')
except Exception as e:
    sg.popup(f'フォーマットとなるExcelブックを開くことが出来ませんでした。\n{e}')
    exit()

In [2]:
layout = [
    [sg.Button('ボタン')]
]

window = sg.Window('test', layout)

while True :
    # イベントを取得
    event, values = window.read()

    if event == sg.WINDOW_CLOSED :
        window.close()
        break

    if event == 'ボタン' :
        event = sg.popup('テスト')
        sg.popup(event)
        continue

In [6]:
# GUIを定義する
layout = [
    [sg.Text('上4つのテキストボックスを入力し、「↓」ボタンを押してください。')],
    [sg.Text('caseDaily:'), sg.Input('', key='-caseDailyPath-'), sg.FileBrowse()],
    [sg.Text('userCaseDaily:'), sg.Input('', key='-userCaseDailyPath-'), sg.FileBrowse()],
    [sg.Text('月間アセスメント実施日:'), sg.Input('', key='-assessmentDay-')],
    [sg.Text('Excelブックの出力先:'), sg.Input('', key='-outputPath-'), sg.FolderBrowse()],
    [sg.Button('↓')],
    [sg.Text('氏名:'), sg.Input('', key='-name-')],
    [sg.Text('受給者証番号:'), sg.Input('', key='-id-')],
    [sg.Text('月間アセスメント担当者名:'), sg.Input('', key='-assessmentAuthor-')],
    [sg.Text('', key='-notification1-')],
    [sg.Text('')],
    [sg.Text('Format'), sg.Input('在宅支援記録表及びアセスメント表（週間・月間）.xlsx', key='-formatPath-', readonly=True), sg.FileBrowse()],
    [sg.Text('Data'), sg.Input('master/master.csv', key='-masterDataPath-', readonly=True), sg.FileBrowse()],
    [sg.Button('実行')],
    [sg.Text('', key='-notification2-')]
]
window = sg.Window('在宅支援記録及びアセスメント表作成支援ツール', layout)

flag = False
verify_flag = False
# イベントを処理
while True :
    # イベントを取得
    event, values = window.read()

    if event == sg.WINDOW_CLOSED :
        flag = False
        window.close()
        break

    if event == '実行': # 実行ボタンが押された時の処理
        flag = True
        val1 = values['-caseDailyPath-']
        val2 = values['-userCaseDailyPath-']
        val3 = values['-outputPath-']

        #テキストボックスおよびファイル検査

        verify_flag = True

        #10. caseDailyが空欄でない
        #11. userCaseDailyが空欄でない
        #12. 出力先が空欄でない
        if val1 == '' or val2 == '' or val3 == '' :
            verify_flag = False
            empty_list = []
            if val1 == '' : empty_list.append('caseDaily欄')
            if val2 == '' : empty_list.append('userCaseDaily欄')
            if val3 == '' : empty_list.append('Excelブックの出力先')

            empty_str = ''
            for i, j in enumerate(empty_list) :
                if i > 0 : 
                    if i == len(empty_list) - 1 :empty_str += 'および' 
                    else : empty_str += '、'
                empty_str += j
            sg.popup(f'{empty_str}は必須項目です。空欄にはできません。')

        if not verify_flag :continue

        #3. caseDailyにcaseDailyのファイルが入っている
        #4. caseDailyにcsvファイルが入っている
        #1. userCaseDailyにuserCaseDailyのファイルが入っている
        #2. userCaseDailyにcsvファイルが入っている
        filename1 = val1.split('/').pop()
        filename2 = val2.split('/').pop()

        errMsg_list = []
        if len(filename1) < 4 :
            verify_flag = False
            errMsg_list.append('caseDaily欄に指定した値が有効ではありません。')
        elif filename1[-4:] != '.csv' :
            verify_flag = False
            errMsg_list.append('caseDaily欄に指定したファイルはCSVではありません。')
        elif len(filename1) < 19 :
            verify_flag = False
            errMsg_list.append('caseDaily欄に指定した値が有効ではありません。')
        elif filename1[:9] != 'caseDaily':
            verify_flag = False
            errMsg_list.append('caseDaily欄に指定した値が有効ではありません。')
            
        if len(filename2) < 4 :
            verify_flag = False
            errMsg_list.append('userCaseDaily欄に指定した値が有効ではありません。')
        elif filename2[-4:] != '.csv' :
            verify_flag = False
            errMsg_list.append('userCaseDaily欄に指定したファイルはCSVではありません。')
        elif len(filename2) < 23 :
            verify_flag = False
            errMsg_list.append('userCaseDaily欄に指定した値が有効ではありません。')
        elif filename2[:13] != 'userCaseDaily' :
            verify_flag = False
            errMsg_list.append('userCaseDaily欄に指定した値が有効ではありません。')

        if not verify_flag :
            errMsg = ''
            for i, j in enumerate(errMsg_list) :
                if i : errMsg += '\n'
                errMsg += j
            sg.popup(errMsg)
            continue
        
        #追加. caseDailyとuserCaseDailyのIDが同じである
        #9. caseDailyとuserCaseDailyの年月が同じである
        filename1_list = filename1.split('_')
        filename2_list = filename2.split('_')

        differ_list = []
        if filename1_list[1] != filename2_list[1] or filename1_list[2] != filename2_list[2] :
            verify_flag = False

            if filename1_list[1] != filename2_list[1] : differ_list.append('ID')
            if filename1_list[2][:6] != filename2_list[2][:6] : differ_list.append('年月')

            differ_str = ''
            for i, j in enumerate(differ_list) :
                if i :differ_str += 'および'
                differ_str += j

        if not verify_flag :
            sg.popup(f'caseDailyとuserCaseDailyで{differ_str}が異なります。')
            continue

        #追加. 出力先が絶対パスである
        if not os.path.isabs(val3) :
            verify_flag = False
            sg.popup('Excelブックの出力先はファイル選択ダイアログから指定してください。')
            continue

        #13. 出力されるxlsxファイルが既に存在する時、上書きするかどうかのダイアログを出す
        #8. 出力されるxlsxファイルのアクセス拒否エラーをTryで渡す
        #   不要. caseDailyおよびuserCaseDailyの各ファイルについてもアクセス拒否エラーをTryで渡す
        namedf = csvConvert(val1)
        name = namedf.columns.values[0]
        saveFilename = f'{name}_{filename1_list[2][:6]}_サービス支援記録.xlsx'
        savePath = f'{val3}\\{saveFilename}'

        if os.path.exists(savePath) :
            result = sg.popup_ok_cancel(f'{savePath}は既に存在します。上書きしますか？',
                                        title='サービス支援記録・簡易作成ツール')
            if result == 'OK' :
                try:
                    wb = openpyxl.load_workbook(savePath)
                except PermissionError as e:
                    verify_flag = False
                    sg.popup('出力先のExcelブックを開くことが出来ませんでした。\
Excelブックを閉じてからやり直してください。')
            else :verify_flag = False

        if not verify_flag :continue

        caseDailyPath = val1
        userCaseDailyPath = val2
        window.close()
        break

#if not flag :exit()

In [6]:
#緊急用　いらない
window.close()

In [5]:
#ファイルを読み込み、エンコードをUTF-8に変換してデータフレームにする
#それぞれのデータフレームの氏名欄に\u3000(全角スペース)がある場合、半角スペースに変換する
caseDaily = csvConvert(caseDailyPath)
userCaseDaily = csvConvert(userCaseDailyPath)

name = caseDaily.columns.values[0]
df = pd.merge(caseDaily, userCaseDaily, on=name, how='left')

In [6]:
#'出欠等'列が'欠席時対応'の行を除去する
droplist = []
for i, j in zip(df.index, df['出欠等']):
    if j == "欠席時対応" :droplist.append(i)
df = df.drop(df.index[droplist])

In [9]:
df.columns

Index(['秋葉原 大夫', '予定', '出欠等', '実績開始時間_x', '実績終了時間_x', '訪問', '送迎(往)',
       '送迎同一敷地内(往)', '送迎(往).1', '送迎同一敷地内(往).1', '食事提供', '医療連携', '施設外支援',
       '移行準備支援体制加算Ⅰ(同行)', '移行準備支援体制加算Ⅱ', '実績記録票備考欄', '日報_x', '出欠等詳細', '確認事項',
       '確認事項対応結果', '確認事項対応状況', '備考_x', '面談対応者', '面談区分', '面談タイトル', '面談詳細',
       '面談備考', '請求', 'プログラム', '体温', '予定開始時間', '予定終了時間', '実績開始時間_y', '実績終了時間_y',
       '睡眠開始時間', '睡眠終了時間', '熟睡感', '起床時の意欲', '睡眠メモ', '昨日昼食', '昨日夕食', '本日朝食',
       '食事メモ', '午前のプログラム', '午前のプログラム詳細', '午後1のプログラム', '午後1のプログラム詳細',
       '午後2のプログラム', '午後2のプログラム詳細', '終日のプログラム', '終日のプログラム詳細', '日報_y', '備考_y'],
      dtype='object')

In [64]:
#strとfloat_nanを同時に判別する関数
def nan_check(val) :
    if type(val) is str :return val
    else :
        if math.isnan(val) :
            return False
        else :return val

#Excelブックを開く
wb = openpyxl.load_workbook('サービス支援記録ーSample Format(張翔).xlsx')

#年月日を指定する
separate_code = df[name][0][4]

#データを取り出す
for i1, i2, i3, i4, i5, i6, i7, i8, i9, i10, i11 in zip(
    df[name], df['実績開始時間_x'], df['実績終了時間_x'], df['実績記録票備考欄'],
    df['午前のプログラム'], df['午前のプログラム詳細'], df['午後1のプログラム'], df['午後1のプログラム詳細'],
    df['日報_x'], df['体温'], df['備考_x']
):
    #データ処理
    place = ''
    if nan_check(i4) == False :
        place = '事業所'
    elif i4 == '在宅支援（届出済）' :
        place = '利用者宅'
    else :place = 'その他'
    
    temperature = ''
    if nan_check(i10) == False:
        temperature = '体温未計測'
    else :
        temperature = str(i10) + '℃'
    
    yyyymm = i1
    yyyymm_list = yyyymm.split(separate_code)
    date = yyyymm_list[0] + '年'
    if int(yyyymm_list[1]) < 10 & len(yyyymm_list[1]) > 1 :
        date += (yyyymm_list[1][1] + '月')
    else :
        date += (yyyymm_list[1] + '月')
    if int(yyyymm_list[2]) < 10 & len(yyyymm_list[2]) > 1  :
        date += (yyyymm_list[2][1] + '日')
    else :
        date += (yyyymm_list[2] + '日')
    
    time = f'{str(i2[:5])}～{str(i3[:5])}'

    work = ''
    if nan_check(i5) :
        work = i5
        for i in [i6, i7, i8] :
            if nan_check(i) : work += '\n' + i
    else :
        work = i9
        work_list = []
        result = re.findall(r'本人.+?「.+?」', work)
        work = result[0][result[0].find('「')+1:-1]

    #備考欄の処理
    remarks = i11
    remarks_list = []
    result = re.findall(r'\d+:\d+', remarks)
    for i in result :
        remarks_list = re.split(r'\d+:\d+', remarks)
    result.insert(0, '')

    remarks = ''
    for i, (j, k) in enumerate(zip(result, remarks_list)) :
        if i and k[0] != ' ' : k = ' ' + k
        remarks = remarks + (j + k[:30] + '・・・・\n')

    #print(i1, i2, i3, i4, i5, i6, i7, i8, i9, i10, i11)

    #print(date, time, place, temperature)
    #print(work)
    #print(i9)
    #print(result)
    #print(remarks)

    #Excelブックのシートをコピーし、データを入力する
    sheet = wb.copy_worksheet(wb['Format'])
    sheet.title = date
    sheet.sheet_view.view = 'pageBreakPreview'

    sheet['B4'] = date
    sheet['G4'] = name
    sheet['B5'] = time
    sheet['G5'] = place
    sheet['A9'] = work
    sheet['A11'] = i9
    sheet['B13'] = temperature
    sheet['A16'] = remarks
    '''
    B4 G4(name) B5 G5 A9 A11 B13 A16
    date name time place work i9 temperature remarks
    '''


In [65]:
#FormatシートとSampleシートを削除する
wb.remove(wb['Format'])
wb.remove(wb['Sample'])

#yyyymm = yyyymm_list[0] + yyyymm_list[1]

#Excelブックを保存する
#wb.save(name + '_' + yyyymm + '_サービス支援記録.xlsx')
if not os.path.isdir(val3) :
    os.makedirs(val3)
wb.save(savePath)

sg.popup(f'{savePath}への出力が完了しました。')

'OK'